<a href="https://colab.research.google.com/github/Jiaye39/TimeSeriesAnalysis/blob/main/Pre_processing_5steps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pre_processing_5steps

In [1]:
import requests
r = requests.get("http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz")
imdb_tgz = r.content

In [2]:
import io
import re
import tarfile

from tqdm.notebook import tqdm

good_file = re.compile(r"^aclImdb/(test|train)/(pos|neg)/.*\.txt$")

with tarfile.open(fileobj=io.BytesIO(r.content), mode="r:gz") as tgz:
    all_members = tgz.getmembers()
    data_files = list(filter(lambda x: x.isfile() and good_file.match(x.name) is not None, all_members))
    for f in tqdm(data_files):
        tgz.extract(f)

  0%|          | 0/50000 [00:00<?, ?it/s]

/tmp/ipython-input-674617911.py:13: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tgz.extract(f)


In [3]:
from sklearn.datasets import load_files
train_data, test_data = load_files("./aclImdb/train", encoding="utf-8"), load_files("./aclImdb/test", encoding="utf-8")

In [4]:
label2txt = {label: txt for label, txt in enumerate(train_data.target_names)}
txt2label = {txt: label for label, txt in label2txt.items()}

In [5]:
type(train_data)

sklearn.utils._bunch.Bunch

In [6]:
X_train, y_train = train_data.data, train_data.target
X_test, y_test = test_data.data, test_data.target

# The data
* `X_train` and `X_test` are `list` of 25000 `str` texts
* `y_train` and `y_test` are `list` of 25000 `int`, either `0` (negative review) or `1` (positive review)


In [7]:
import numpy as np

print("TRAIN data:")
print("class balance: ", np.bincount(y_train))
print()
print("TEST data:")
print("class balance: ", np.bincount(y_train))



TRAIN data:
class balance:  [12500 12500]

TEST data:
class balance:  [12500 12500]


In [8]:
!pip install eli5

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.4/108.4 kB 3.8 MB/s eta 0:00:00


In [9]:
import nltk
nltk.download("punkt")
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [10]:
from nltk.tokenize import word_tokenize

In [11]:
import nltk
nltk.download("stopwords")

!pip install gensim
from gensim.corpora import Dictionary

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 42.4 MB/s eta 0:00:00


In [12]:
import rich

from rich.console import Console
from rich.table import Table

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Preprocessing

In [14]:
def create_table(dictionary: Dictionary, n: int = 10) -> Table:
    table = Table(title=f"Top-{n} Most Frequent Tokens ({d.num_docs} documents, {d.num_pos} tokens, {len(d)} words in dictionary)")

    table.add_column("Token", justify="left", no_wrap=True)
    table.add_column("Corpus Frequency", justify="right")
    table.add_column("% of Tokens", justify="right")
    table.add_column("Document Frequency", justify="right")
    table.add_column("% of Documents", justify="right")

    for token, frequency in d.most_common(n=n):
        percent_tokens = frequency / d.num_pos
        doc_frequency = d.dfs[d.token2id[token]]
        percent_doc = doc_frequency / d.num_docs
        table.add_row(token, str(frequency), f"{percent_tokens:.1%}", str(doc_frequency), f"{percent_doc:.1%}")

    return table

## 1st Pre-Processing: Lowercase

In [15]:
def lowercase(text: str) -> list[str]:
    """Tokenize and preprocess the text.

    Normalize the text to lowercase

    Returns:
        tokens: a list of tokens
    """
    return [x.lower() for x in word_tokenize(text)]

In [16]:
X_train_tokenized = [lowercase(x) for x in tqdm(X_train)]

  0%|          | 0/25000 [00:00<?, ?it/s]

In [17]:
d = Dictionary(X_train_tokenized)

In [18]:
console = Console(record=True, width=80)
t = create_table(d, 20)
console.print(t, justify="center")
console.save_svg("all_vocab.svg", title="Vocabulary")

 Top-20 Most Frequent Tokens (25000 documents, 7065344 tokens, 111720 words in  
                                  dictionary)                                   
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Token ┃ Corpus Frequency ┃ % of Tokens ┃ Document Frequency ┃ % of Documents ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ the   │           334840 │        4.7% │              24791 │          99.2% │
│ ,     │           275887 │        3.9% │              24054 │          96.2% │
│ .     │           236573 │        3.3% │              24685 │          98.7% │
│ and   │           163477 │        2.3% │              24159 │          96.6% │
│ a     │           162304 │        2.3% │              24173 │          96.7% │
│ of    │           145404 │        2.1% │              23721 │          94.9% │
│ to    │           135153 │        1.9% │              23459 │          93.8% │
│ is    │           110284 │        1.6% │              22522 │          90.1% │
│ /     │           102115 │        1.4% │              14711 │          58.8% │
│ >     │           102036 │        1.4% │              14681 │          58.7% │
│ <     │           101971 │        1.4% │              14671 │          58.7% │
│ br    │           101871 │        1.4% │              14666 │          58.7% │
│ it    │            95024 │        1.3% │              22207 │          88.8% │
│ in    │            93197 │        1.3% │              22004 │          88.0% │
│ i     │            86747 │        1.2% │              19825 │          79.3% │
│ this  │            75566 │        1.1% │              22607 │          90.4% │
│ that  │            73011 │        1.0% │              20292 │          81.2% │
│ 's    │            62167 │        0.9% │              17895 │          71.6% │
│ was   │            50383 │        0.7% │              16377 │          65.5% │
│ as    │            46834 │        0.7% │              16103 │          64.4% │
└───────┴──────────────────┴─────────────┴────────────────────┴────────────────┘

## 2nd Preprocessing: lowercase + stopping

In [19]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
from nltk.corpus import stopwords
stopwords = set(stopwords.words("english"))

In [21]:
print(stopwords)

{'mightn', 'me', 'these', "don't", 'if', "we're", 'was', "won't", 'the', "that'll", "we've", "you're", 'all', "they're", 'own', 'yours', 'by', 'it', 'ourselves', "doesn't", 'won', 'she', "needn't", 'whom', 'up', 'does', "couldn't", 'itself', 'below', "she'd", 'those', "i'll", "wasn't", 'in', 'to', 'having', 'him', 'your', 'are', "he'll", 'were', 'against', 'or', 'further', 'before', 'at', "he'd", 'ours', 'hers', "it'd", 'here', 'i', 'nor', 'doesn', 'm', 'yourselves', "weren't", 'themselves', 'needn', 'because', "they've", 'above', 'them', "it'll", 'his', 'other', "they'll", 'have', 'do', 'been', 're', "i'm", 'that', "you've", 'wouldn', 'some', 'very', 'herself', 'isn', 'doing', 'through', 'an', 'but', 'ma', 'can', 'for', 'more', 'on', 'myself', 'so', 'shan', 'about', 'be', 'did', 'a', 'no', "you'd", "she'll", 'd', 'any', 'ain', 'our', "you'll", 'each', 'my', "i'd", 'hasn', 'as', 't', "he's", 'their', 'you', 'only', 'hadn', "aren't", 'too', 'y', "mustn't", 'this', 'couldn', 'll', 'off',

In [22]:
def lower_stop(text: str, stopwords: set[str]) -> list[str]:
    """Tokenize and preprocess the text.

    Normalize the text to lowercase and remove stopwords.

    Returns:
        tokens: a list of tokens
    """
    tokens = lowercase(text)
    return [x for x in tokens if x not in stopwords]

In [23]:
X_train_tokenized = [lower_stop(text=x, stopwords=stopwords) for x in tqdm(X_train)]

  0%|          | 0/25000 [00:00<?, ?it/s]

In [24]:
console = Console(record=True, width=120)
d = Dictionary(X_train_tokenized)
t = create_table(d, 20)
console.print(t)


 Top-20 Most Frequent Tokens (25000 documents, 4280399 tokens, 111572 words in  
                                  dictionary)                                   
┏━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Token ┃ Corpus Frequency ┃ % of Tokens ┃ Document Frequency ┃ % of Documents ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ ,     │           275887 │        6.4% │              24054 │          96.2% │
│ .     │           236573 │        5.5% │              24685 │          98.7% │
│ /     │           102115 │        2.4% │              14711 │          58.8% │
│ >     │           102036 │        2.4% │              14681 │          58.7% │
│ <     │           101971 │        2.4% │              14671 │          58.7% │
│ br    │           101871 │        2.4% │              14666 │          58.7% │
│ 's    │            62167 │        1.5% │              17895 │          71.6% │
│ movie │            42595 │        1.0% │              15101 │          60.4% │
│ film  │            38758 │        0.9% │              13719 │          54.9% │
│ )     │            36175 │        0.8% │              13495 │          54.0% │
│ (     │            35397 │        0.8% │              13361 │          53.4% │
│ n't   │            33380 │        0.8% │              14839 │          59.4% │
│ ''    │            33129 │        0.8% │              11488 │          46.0% │
│ ``    │            32857 │        0.8% │              11507 │          46.0% │
│ one   │            25893 │        0.6% │              13893 │          55.6% │
│ !     │            24560 │        0.6% │               8595 │          34.4% │
│ like  │            19745 │        0.5% │              11477 │          45.9% │
│ ?     │            16088 │        0.4% │               7388 │          29.6% │
│ good  │            14658 │        0.3% │               9439 │          37.8% │
│ would │            13387 │        0.3% │               8573 │          34.3% │
└───────┴──────────────────┴─────────────┴────────────────────┴────────────────┘

## 3rd Preprocessing: accept only alphabetical tokens

In [25]:
def lower_stop_only_alpha(text: str, stopwords: set[str]) -> list[str]:
    """Tokenize and preprocess the text.

    Normalize the text to lowercase, remove stopwords, remove tokens
    that are not entirely made of letters, remove tokens with only 1 character.

    Returns:
        tokens: a list of tokens
    """
    own_stopwords = {"br"}
    tokens = lower_stop(text=text, stopwords=stopwords | own_stopwords)
    return [x for x in tokens if x.isalpha() and len(x) > 1]

In [26]:
X_train_tokenized = [lower_stop_only_alpha(text=x, stopwords=stopwords) for x in tqdm(X_train)]

  0%|          | 0/25000 [00:00<?, ?it/s]

In [27]:
d = Dictionary(X_train_tokenized)

In [28]:
console = Console(record=True, width=120)
t = create_table(d, 20)
console.print(t)
console.save_svg("alpha_vocab.svg", title="Vocabulary")

  Top-20 Most Frequent Tokens (25000 documents, 2860910 tokens, 71325 words in   
                                   dictionary)                                   
┏━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Token  ┃ Corpus Frequency ┃ % of Tokens ┃ Document Frequency ┃ % of Documents ┃
┡━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ movie  │            42595 │        1.5% │              15101 │          60.4% │
│ film   │            38758 │        1.4% │              13719 │          54.9% │
│ one    │            25893 │        0.9% │              13893 │          55.6% │
│ like   │            19745 │        0.7% │              11477 │          45.9% │
│ good   │            14658 │        0.5% │               9439 │          37.8% │
│ would  │            13387 │        0.5% │               8573 │          34.3% │
│ even   │            12609 │        0.4% │               8397 │          33.6% │
│ time   │            11899 │        0.4% │               8327 │          33.3% │
│ really │            11702 │        0.4% │               7609 │          30.4% │
│ story  │            11583 │        0.4% │               7457 │          29.8% │
│ see    │            11278 │        0.4% │               7982 │          31.9% │
│ much   │             9611 │        0.3% │               7004 │          28.0% │
│ well   │             9498 │        0.3% │               6937 │          27.7% │
│ could  │             9377 │        0.3% │               6819 │          27.3% │
│ get    │             9235 │        0.3% │               6730 │          26.9% │
│ people │             9168 │        0.3% │               6167 │          24.7% │
│ also   │             9099 │        0.3% │               6417 │          25.7% │
│ bad    │             8982 │        0.3% │               5762 │          23.0% │
│ great  │             8935 │        0.3% │               6255 │          25.0% │
│ first  │             8882 │        0.3% │               6309 │          25.2% │
└────────┴──────────────────┴─────────────┴────────────────────┴────────────────┘

## 4th Preprocessing - Stemming

* Reduces the vocabulary by reducing plurals, conjugations to their root form
* "making", "makes", "made" are transformed into "make"
* "cats", "cat" transform into "cat"
* **but** "movie" is transformed into "movi"

In [29]:
from nltk.stem import PorterStemmer


In [30]:
porter = PorterStemmer()

In [31]:
porter.stem("cats")

'cat'

In [32]:
porter.stem("making")

'make'

In [33]:
porter.stem("movie")

'movi'

In [34]:
def lower_stop_only_alpha_stem(text: str, stopwords: list[str]) -> list[str]:
    tokens = lower_stop_only_alpha(text, stopwords)
    return [porter.stem(x) for x in tokens]

In [35]:
X_train_tokenized = [lower_stop_only_alpha_stem(text=x, stopwords=stopwords) for x in tqdm(X_train)]

  0%|          | 0/25000 [00:00<?, ?it/s]

In [36]:
d = Dictionary(X_train_tokenized)


In [37]:
console = Console(record=True, width=120)
t = create_table(d, 20)
console.print(t)
console.save_svg("stem.svg", title="Vocabulary")

   Top-20 Most Frequent Tokens (25000 documents, 2860910 tokens, 48426 words in   
                                   dictionary)                                    
┏━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Token   ┃ Corpus Frequency ┃ % of Tokens ┃ Document Frequency ┃ % of Documents ┃
┡━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ movi    │            50022 │        1.7% │              16039 │          64.2% │
│ film    │            46628 │        1.6% │              14710 │          58.8% │
│ one     │            26816 │        0.9% │              14115 │          56.5% │
│ like    │            22248 │        0.8% │              12391 │          49.6% │
│ time    │            15226 │        0.5% │               9936 │          39.7% │
│ good    │            14866 │        0.5% │               9540 │          38.2% │
│ make    │            14530 │        0.5% │               9485 │          37.9% │
│ get     │            14042 │        0.5% │               8962 │          35.8% │
│ see     │            13881 │        0.5% │               9274 │          37.1% │
│ charact │            13873 │        0.5% │               8169 │          32.7% │
│ watch   │            13685 │        0.5% │               9058 │          36.2% │
│ would   │            13387 │        0.5% │               8573 │          34.3% │
│ even    │            12856 │        0.4% │               8529 │          34.1% │
│ stori   │            12732 │        0.4% │               7836 │          31.3% │
│ realli  │            11702 │        0.4% │               7609 │          30.4% │
│ scene   │            10306 │        0.4% │               6436 │          25.7% │
│ well    │             9874 │        0.3% │               7027 │          28.1% │
│ look    │             9720 │        0.3% │               6734 │          26.9% │
│ show    │             9676 │        0.3% │               5328 │          21.3% │
│ much    │             9611 │        0.3% │               7004 │          28.0% │
└─────────┴──────────────────┴─────────────┴────────────────────┴────────────────┘

## 5th Preprocessing - NGrams

* 2-grams are made of 2 consecutive tokens in the text
* "the cat is blue" has 2 grams `["the", "cat"], ["cat", "is"], ["is", "blue"]`

In [38]:
from more_itertools import windowed

In [39]:
def get_ngrams(tokens: list[str], n: int) -> list[str]:
    return [" ".join(x) for x in windowed(tokens, n=n)]

In [40]:
tokens = lower_stop("I live in new york city", [])
print(tokens)

['i', 'live', 'in', 'new', 'york', 'city']


In [41]:
print(get_ngrams(tokens, n=2))

['i live', 'live in', 'in new', 'new york', 'york city']


In [42]:
print(get_ngrams(tokens, n=3))

['i live in', 'live in new', 'in new york', 'new york city']


In [43]:
def lower_stop_only_alpha_stem_ngrams(text: str, stopwords: list[str], ngrams: int) -> list[str]:
    tokens = lower_stop_only_alpha_stem(text, stopwords)
    n_grams = get_ngrams(tokens, ngrams)
    return n_grams


In [44]:
lower_stop_only_alpha_stem_ngrams("I live in the middle of new york city.", stopwords=stopwords, ngrams=3)

['live middl new', 'middl new york', 'new york citi']

In [45]:
X_train_tokenized = [lower_stop_only_alpha_stem_ngrams(x, stopwords, 2) for x in tqdm(X_train)]

  0%|          | 0/25000 [00:00<?, ?it/s]

In [46]:
d = Dictionary(X_train_tokenized)

In [47]:
console = Console(record=True, width=120)
t = create_table(d, 20)
console.print(t)
console.save_svg("bigrams.svg", title="Vocabulary")

     Top-20 Most Frequent Tokens (25000 documents, 2835910 tokens, 1468424 words in      
                                       dictionary)                                       
┏━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Token          ┃ Corpus Frequency ┃ % of Tokens ┃ Document Frequency ┃ % of Documents ┃
┡━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ look like      │             1793 │        0.1% │               1553 │           6.2% │
│ watch movi     │             1526 │        0.1% │               1380 │           5.5% │
│ ever seen      │             1267 │        0.0% │               1202 │           4.8% │
│ special effect │             1110 │        0.0% │                971 │           3.9% │
│ even though    │             1041 │        0.0% │                952 │           3.8% │
│ movi like      │              987 │        0.0% │                911 │           3.6% │
│ movi ever      │              985 │        0.0% │                916 │           3.7% │
│ see movi       │              975 │        0.0% │                899 │           3.6% │
│ main charact   │              924 │        0.0% │                816 │           3.3% │
│ one best       │              908 │        0.0% │                841 │           3.4% │
│ wast time      │              895 │        0.0% │                831 │           3.3% │
│ make movi      │              878 │        0.0% │                811 │           3.2% │
│ year old       │              869 │        0.0% │                762 │           3.0% │
│ good movi      │              781 │        0.0% │                732 │           2.9% │
│ watch film     │              780 │        0.0% │                728 │           2.9% │
│ seem like      │              761 │        0.0% │                706 │           2.8% │
│ horror movi    │              741 │        0.0% │                566 │           2.3% │
│ want see       │              726 │        0.0% │                689 │           2.8% │
│ new york       │              724 │        0.0% │                566 │           2.3% │
│ much better    │              720 │        0.0% │                680 │           2.7% │
└────────────────┴──────────────────┴─────────────┴────────────────────┴────────────────┘

In [48]:
t = create_table(d, 200)
console.print(t)

Top-200 Most Frequent Tokens (25000 documents, 2835910 tokens, 1468424 words in dictionary)
┏━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Token            ┃ Corpus Frequency ┃ % of Tokens ┃ Document Frequency ┃ % of Documents ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ look like        │             1793 │        0.1% │               1553 │           6.2% │
│ watch movi       │             1526 │        0.1% │               1380 │           5.5% │
│ ever seen        │             1267 │        0.0% │               1202 │           4.8% │
│ special effect   │             1110 │        0.0% │                971 │           3.9% │
│ even though      │             1041 │        0.0% │                952 │           3.8% │
│ movi like        │              987 │        0.0% │                911 │           3.6% │
│ movi ever        │              985 │        0.0% │                916 │           3.7% │
│ see movi         │              975 │        0.0% │                899 │           3.6% │
│ main charact     │              924 │        0.0% │                816 │           3.3% │
│ one best         │              908 │        0.0% │                841 │           3.4% │
│ wast time        │              895 │        0.0% │                831 │           3.3% │
│ make movi        │              878 │        0.0% │                811 │           3.2% │
│ year old         │              869 │        0.0% │                762 │           3.0% │
│ good movi        │              781 │        0.0% │                732 │           2.9% │
│ watch film       │              780 │        0.0% │                728 │           2.9% │
│ seem like        │              761 │        0.0% │                706 │           2.8% │
│ horror movi      │              741 │        0.0% │                566 │           2.3% │
│ want see         │              726 │        0.0% │                689 │           2.8% │
│ new york         │              724 │        0.0% │                566 │           2.3% │
│ much better      │              720 │        0.0% │                680 │           2.7% │
│ horror film      │              708 │        0.0% │                534 │           2.1% │
│ like movi        │              701 │        0.0% │                668 │           2.7% │
│ first time       │              692 │        0.0% │                656 │           2.6% │
│ film like        │              688 │        0.0% │                634 │           2.5% │
│ bad movi         │              683 │        0.0% │                603 │           2.4% │
│ make film        │              668 │        0.0% │                623 │           2.5% │
│ movi one         │              660 │        0.0% │                638 │           2.6% │
│ year ago         │              642 │        0.0% │                614 │           2.5% │
│ see film         │              628 │        0.0% │                590 │           2.4% │
│ feel like        │              621 │        0.0% │                587 │           2.3% │
│ take place       │              610 │        0.0% │                557 │           2.2% │
│ low budget       │              602 │        0.0% │                536 │           2.1% │
│ worst movi       │              594 │        0.0% │                560 │           2.2% │
│ film ever        │              578 │        0.0% │                532 │           2.1% │
│ bad guy          │              577 │        0.0% │                454 │           1.8% │
│ pretti much      │              575 │        0.0% │                532 │           2.1% │
│ one thing        │              556 │        0.0% │                528 │           2.1% │
│ movi would       │              553 │        0.0% │                532 │           2.1% │
│ film one         │              549 │        0.0% │                529 │           2.1% │
│ pretti good      │              538 │     

# Machine Learning

* Turn the 2-step process into a Pipeline
* Hyperparameters:
  * `C` of logistic regression
  * `analyzer` of vectorizer, to select the text pre-processing
  * `ngram_range` of vectorizer, to select ngrams
* Use GridSearchCV to identify the best hyperparameters

In [49]:
import nltk
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
import numpy as np

from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report

In [51]:
stop_words = set(stopwords.words("english"))
porter = PorterStemmer()

def stop(text: str) -> list[str]:
    tokens = word_tokenize(text)
    return [x for x in tokens if x not in stop_words]

def stop_stem(text: str) -> list[str]:
    tokens = stop(text)
    return [porter.stem(x) for x in tokens]

In [52]:
pipe = make_pipeline(
    CountVectorizer(lowercase=True), LogisticRegression(max_iter=10000)
)

param_grid = {
    "logisticregression__C": np.logspace(-2, 2, 5),
    "countvectorizer__ngram_range": [(1, 1), (1, 2)],
    "countvectorizer__analyzer": [stop, stop_stem]
}

grid = GridSearchCV(pipe, param_grid=param_grid, verbose=4)

In [ ]:
grid.fit(X_train, y_train)